In [ ]:
import pandas as pd
import glob
import os
import shutil
import re

from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg
from datetime import datetime

In [ ]:
entry_directory = "Raw"
prepared_directory = "Prepared"
organised_directory = "Organised"

In [ ]:
def calculate_sums(file_path):

    node_data_df = pd.read_csv(file_path, dtype = {'timestamp' : 'string', 'PID' : 'int', 'CPU': 'float64', 'RAM': 'float64'})
    node_data_df['timestamp'] = node_data_df['timestamp'].apply(lambda x: x if len(x.split(".")) > 1 else x + ".000000" )
    node_data_df['epoch'] = node_data_df['timestamp'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") - datetime(1970, 1, 1)).total_seconds())
    min_timestamp = node_data_df['epoch'].min()
    node_data_df['epoch'] = node_data_df['epoch'].apply(lambda x: x - min_timestamp)
    node_data_df.drop('PID', axis='columns', inplace=True)
    node_data_df.drop('timestamp', axis='columns', inplace=True)
    node_data_df = node_data_df.groupby("epoch").sum()

    return node_data_df


for root, _, files in os.walk(f".\{entry_directory}"):
        for file in files:
            full_path = os.path.join(root, file)
            if re.search("\d\d_\d\d_\d\d\d\d_\d\d_\d\d_\d\d.csv$", full_path):
                if os.path.exists(full_path.replace(entry_directory, prepared_directory)):
                    continue
                calculated_dataframe_to_save = calculate_sums(full_path)
                if not os.path.exists(root.replace(entry_directory, prepared_directory)):
                    os.makedirs(root.replace(entry_directory, prepared_directory))
                calculated_dataframe_to_save.to_csv(full_path.replace(entry_directory, prepared_directory))

In [ ]:
root = ".\Prepared"
for functionTypeDirectory in os.listdir(root):
    if os.path.isdir(os.path.join(root, functionTypeDirectory)):
        for functionDirectory in os.listdir(os.path.join(root, functionTypeDirectory)):
            for numberOfNodesDirectory in os.listdir(os.path.join(root, functionTypeDirectory, functionDirectory)):
                destinationPath =  os.path.join(root, functionTypeDirectory, functionDirectory, numberOfNodesDirectory, 'source-data')
                for nodeDirectory in os.listdir(destinationPath):
                    path =  os.path.join(destinationPath, nodeDirectory)
                    all_files = glob.glob(path + "/*.csv")
                    dfs = []
                    i = 1
                    for filename in all_files:
                        df = pd.read_csv(filename)
                        dfs.append(df)
                        if df['RAM'].mean() >= 2 and df['CPU'].head(15).mean() > 3:
                            if not os.path.exists(os.path.join(destinationPath.replace("Prepared", "Organised"), str(i))):
                                os.makedirs(os.path.join(destinationPath.replace("Prepared", "Organised"), str(i)))
                            shutil.copyfile(filename, os.path.join(destinationPath.replace("Prepared", "Organised"), str(i), os.path.basename(filename)))
                        i += 1


In [ ]:
def join_files(entry_dir, category):
    directory = f"./{entry_dir}/{category}"
    udf_dirs = os.listdir(directory)
    label = category
    result_df = pd.DataFrame()
    snapshot = 0
    for udf in udf_dirs:
        for root, _, files in os.walk(f"./{directory}/{udf}"):
            for file in files:
                full_path = os.path.join(root, file)
                node_data_df = pd.read_csv(full_path)
                node_data_df['snapshot'] = snapshot
                snapshot += 1
                node_data_df["label"] = label
                node_data_df["udf"] = udf
                result_df = pd.concat([result_df,node_data_df])
    result_df[["snapshot", "label", "udf", "epoch", "CPU", "RAM"]].to_csv(f"{directory}/joined_{category}.csv", index = False)

labels = ['aggregation', 'filtration', 'filtration-aggregation', 'filtration-aggregation-join', 'filtration-join']

for label in labels:
    join_files(organised_directory, label)

In [ ]:
def smooth_values(entry_dir, category, window_value,):
    # reading file
    path_to_df = f"./{entry_dir}/{category}/joined_{category}.csv"
    df = pd.read_csv(path_to_df)
    cpu_df = df.groupby('snapshot')['CPU'].rolling(window=window_value, min_periods = 1).mean().to_frame()
    ram_df = df.groupby('snapshot')['RAM'].rolling(window=window_value, min_periods = 1).mean().to_frame()
    df['CPU'] = cpu_df.reset_index()['CPU'] 
    df['CPU'] = df['CPU'].round(2)
    df['RAM'] = ram_df.reset_index()['RAM']
    df['RAM'] = df['RAM'].round(2)
    df['epoch'] = df['epoch'].round(3)
    df.fillna(0).to_csv(f"./{entry_dir}/{category}/{window_value}_smooth_{category}.csv", index = False)

for label in labels:
    smooth_values(organised_directory, label, 6)

In [ ]:
%matplotlib inline
joined_aggregation = pd.read_csv("Organised/aggregation/joined_aggregation.csv")
joined_aggregation = joined_aggregation[joined_aggregation["snapshot"] == 1]

#(ggplot(joined_aggregation)         # defining what data to use
#  + aes(x='epoch', y = "CPU")    # defining what variable to use
#  + geom_line(size=0.5, color = "red") # defining the type of plot to use
# )

In [ ]:
joined_aggregation_smooth = pd.read_csv("Organised/aggregation/6_smooth_aggregation.csv")
joined_aggregation_smooth = joined_aggregation_smooth[joined_aggregation_smooth["snapshot"] == 1]

(ggplot(joined_aggregation_smooth)         # defining what data to use
 + aes(x='epoch', y = "CPU")    # defining what variable to use
 + geom_line(size=0.5, color = "red") # defining the type of plot to use
)